In [1]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git
!pip install datasets -q
!pip install peft -q

ERROR: git+https://github.com/huggingface/transformers.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


## Dataset

In [2]:

from datasets import load_dataset

dataset_id = "HuggingFaceM4/VQAv2"
dataset = load_dataset(dataset_id, split="train")

cols_remove = [
    "question_type", 
    "answers", 
    "answer_type",
    "image_id", 
    "question_id"
]

dataset = dataset.remove_columns(cols_remove)
dataset = dataset.train_test_split(test_size=0.1)

dataset_train = dataset["train"]
dataset_test = dataset["test"]


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for HuggingFaceM4/VQAv2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HuggingFaceM4/VQAv2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


## Model

In [3]:
# Specify the model. Note the "pt" suffix.
model_id = "google/paligemma-3b-pt-224"

In [4]:
import torch
from transformers import (
    BitsAndBytesConfig,
    PaliGemmaProcessor, 
    PaliGemmaForConditionalGeneration,
    Trainer,
    TrainingArguments    
)

# Initialise the processor
processor = PaliGemmaProcessor.from_pretrained(model_id)


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [5]:

from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [6]:
# Use BitsAndBytes Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map= {"":0}
)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


In [9]:
# model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to("cuda")
# for param in model.vision_tower.parameters():
#     param.requires_grad = False

# for param in model.multi_modal_projector.parameters():
#     param.requires_grad = True

In [10]:
# args = TrainingArguments(
#     output_dir=".output",
#     num_train_epochs=2,                            
#     per_device_train_batch_size=16,                
#     gradient_accumulation_steps=4,                 
#     warmup_steps=2,                                
#     learning_rate=2e-5,                            
#     weight_decay=1e-6,                             
#     adam_beta2=0.999,                              
#     logging_steps=100,                             
#     save_strategy="steps",                         
#     save_steps=1000,                               
#     save_total_limit=1,                            
#     bf16=True,                                     
#     report_to=["tensorboard"],                                                 
#     # optim="adamw_hf",
#     optim="paged_adamw_8bit",
#     dataloader_pin_memory=False,
#     remove_unused_columns=False
# )

args=TrainingArguments(
    output_dir=".output",
        num_train_epochs=2,
        remove_unused_columns=False,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        learning_rate=2e-5,
        weight_decay=1e-6,
        adam_beta2=0.999,
        logging_steps=100,
        optim="adamw_hf",
        # optim="paged_adamw_8bit",
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=1,
        bf16=True,
        report_to=["tensorboard"],
        dataloader_pin_memory=False
        
    )

In [11]:
device = "cuda"
image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
def collate_fn(examples):
  texts = ["answer " + example["question"] + "\n" + example['multiple_choice_answer'] for example in examples]
  images = [example["image"].convert("RGB") for example in examples]
  tokens = processor(text=texts, images=images,
                    return_tensors="pt", padding="longest",
                    tokenize_newline_separately=False)
  labels = tokens["input_ids"].clone()
  labels[labels == processor.tokenizer.pad_token_id] = -100
  labels[labels == image_token] = -100
  tokens["labels"] = labels
  tokens["pixel_values"] = tokens["pixel_values"].to(model.dtype).to(device)
  tokens = tokens.to(model.dtype).to(device)
  return tokens

In [12]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=collate_fn
)

In [13]:
trainer.train()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  war

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# Assuming you have a small batch of examples for testing
test_examples = [dataset_train[i] for i in range(2)]
collated_data = collate_fn(test_examples)
print({k: v.dtype for k, v in collated_data.items()})  # Check the dtypes of all tensors
